<a href="https://colab.research.google.com/github/shashank-r97/NLP/blob/master/Style/PureCNN_Glove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

In [0]:
#!pip install -U imbalanced-learn

In [3]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential 
from keras.layers import Input, Masking
from keras.layers import Dense, Input, Reshape
from keras.layers import Dropout, Activation
from keras.layers import Dense, GlobalAveragePooling1D, Activation
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM, SimpleRNN
from keras.layers import Embedding
import keras.backend as K 
from imblearn.combine import SMOTETomek
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.layers.convolutional import Conv1D
from keras.layers.merge import concatenate
from keras.layers.convolutional import MaxPooling1D
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.phrases import Phraser, Phrases

Using TensorFlow backend.


In [4]:
%%time
df = pd.read_csv('/content/drive/Shared drives/DSO 560 NLP Project/data.csv')
df = df[df['attribute_name'] == 'style'].drop(columns = ['attribute_name'])
df['attribute_value'] = df['attribute_value'].apply(lambda x: 'businesscasual' if x == 'business casual' else x)

CPU times: user 429 ms, sys: 71.5 ms, total: 500 ms
Wall time: 537 ms


In [5]:
df.head()

,product_id,brand,product_full_name,description,brand_category,attribute_value,details
0,01DPGV4YRP3Z8J85DASGZ1Y99W,frame,les second medium noir,"minimal , modern styling meet refined luxury l...",accessory,casual,NaN
5,01DPH1DEN9G2WM7WAMJMD0A9W4,j crew,tie waist shirtdress stripe,take classic button silhouette turn ultra flat...,dressesandjumpsuits,casual,NaN
12,01E2KYW52BAG606GQ7A9H5R0KD,alo,interval microfleece pullover hoodie,articulate seam extra wide rib hem create shap...,unknown,casual,"xs 0 2 , s 4 6 , m 8 10 , l 12 14"
13,01DT513RRYT3SKH6X25G5VCH6B,chlo,leather ankle boot,heel measure approximately 55 mm 2 inch 30 mm ...,shoe boots ankle,androgynous,"fit small size , size large normal available s..."
20,01E2KM0KW6NB1JKMZVRXR6H8G2,alo,stadium quarter zip hoodie,supersoft hoodie design elastic hem cuff perfe...,unknown,casual,"xs 0 2 , s 4 6 , m 8 10 , l 12 14"


In [0]:
df['text'] = (df['brand'] + ' ' + df['product_full_name'] + ' ' + df['description'] + ' ' + df['brand_category']).apply(str)

In [0]:
X = df['text'].values
y = pd.get_dummies(df['attribute_value'])
label_list = y.columns
y = y.values

In [0]:
tokenizer = Tokenizer(num_words=5000, oov_token="UNKNOWN_TOKEN")
tokenizer.fit_on_texts(X)

In [0]:
X = tokenizer.texts_to_sequences(X)

In [0]:
length = max([len(s.split()) for s in df['text']])
vocab_size = len(tokenizer.word_index) + 1

In [0]:
X = pad_sequences(X, maxlen=length, padding='post')

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)

In [0]:
resampler = SMOTETomek(sampling_strategy = 'auto')
X_train, y_train = resampler.fit_resample(X_train, y_train)

In [0]:
num_classes = y.shape[1]

In [15]:
def load_glove_vectors():
    embeddings_index = {}
    with open('/content/drive/My Drive/NLP/glove.6B.100d.txt') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    print('Loaded %s word vectors.' % len(embeddings_index))
    return embeddings_index


embeddings_index = load_glove_vectors()

Loaded 400000 word vectors.


In [0]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: # check that it is an actual word that we have embeddings for
        embedding_matrix[i] = embedding_vector

In [0]:
def define_model():
    
    model = Sequential()
    model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=length, trainable=False))
    #model.add(Masking(mask_value=0.0)) # masking layer, masks any words that don't have an embedding as 0s.
    model.add(Conv1D(filters=128, kernel_size=10, padding='same', activation = 'relu'))
    model.add(Conv1D(filters=128, kernel_size=10, padding='same', activation = 'relu'))
    model.add(MaxPooling1D(pool_size=2))

    model.add(Conv1D(filters=64, kernel_size=10, padding='same', activation = 'relu'))
    model.add(Conv1D(filters=64, kernel_size=10, padding='same', activation = 'relu'))
    model.add(MaxPooling1D(pool_size=2))

    model.add(Conv1D(filters = num_classes, kernel_size= 38, activation = 'relu'))
    model.add(Conv1D(filters = num_classes, kernel_size= 1))
    model.add(Reshape((num_classes,)))
    model.add(Activation('sigmoid'))
    
    # Compile the model
    model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

def define_model():
    
    model = Sequential()
    model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=length, trainable=False))
    #model.add(Masking(mask_value=0.0)) # masking layer, masks any words that don't have an embedding as 0s.
    LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.15, activation = 'relu')
    LSTM(128, return_sequences=False, dropout=0.2, recurrent_dropout=0.15, activation = 'relu')
    model.add(Flatten())
    model.add(Dense(16, activation = 'relu'))
    model.add(Dense(num_classes, activation='sigmoid'))
    
    # Compile the model
    model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [18]:
model = define_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 155, 100)          611400    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 155, 128)          128128    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 155, 128)          163968    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 77, 128)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 77, 64)            81984     
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 77, 64)            41024     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 38, 64)           

In [19]:
model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=512)

Train on 20350 samples, validate on 5088 samples
Epoch 1/10
20350/20350 [==============================] - 4s 201us/step - loss: 0.5688 - accuracy: 0.7325 - val_loss: 0.5084 - val_accuracy: 0.7942
Epoch 2/10
20350/20350 [==============================] - 1s 74us/step - loss: 0.4652 - accuracy: 0.7630 - val_loss: 0.4534 - val_accuracy: 0.8187
Epoch 3/10
20350/20350 [==============================] - 2s 74us/step - loss: 0.3650 - accuracy: 0.8868 - val_loss: 0.4357 - val_accuracy: 0.9091
Epoch 4/10
20350/20350 [==============================] - 2s 75us/step - loss: 0.3056 - accuracy: 0.9091 - val_loss: 0.4382 - val_accuracy: 0.9091
Epoch 5/10
20350/20350 [==============================] - 2s 74us/step - loss: 0.2877 - accuracy: 0.9092 - val_loss: 0.4252 - val_accuracy: 0.9091
Epoch 6/10
20350/20350 [==============================] - 1s 74us/step - loss: 0.2800 - accuracy: 0.9099 - val_loss: 0.4249 - val_accuracy: 0.9083
Epoch 7/10
20350/20350 [==============================] - 1s 74us/st

In [20]:
model.evaluate(X_test, y_test)

1089/1089 [==============================] - 0s 174us/step


[0.3190883643261548, 0.9063360691070557]

In [0]:
results = model.predict(X)

In [0]:
results_df = pd.DataFrame()
results_mask = results > 0.15
for i in range(len(label_list)):
    results_df[label_list[i]] = results_mask[:,i]
    results_df[label_list[i]] = results_df[label_list[i]].apply(int)

In [29]:
results_df['sum']  = 0
for key in label_list:
    results_df['sum'] = results_df['sum'] + results_df[key]
(results_df['sum'] == 0).sum()

768

In [24]:
results_df = results_df.drop(columns = ['sum'])
for key in label_list:
    results_df[key] = results_df[key].apply(lambda x: key if x == 1 else '')

results_df['attribute_value'] = ''
for key in label_list:
    results_df['attribute_value'] = results_df['attribute_value'] + ' ' + results_df[key]

results_df['attribute_value'] = results_df['attribute_value'].apply(lambda x: ', '.join(x.split()))
results_df.head()

,androgynous,athleisure,boho,businesscasual,casual,classic,edgy,glam,modern,retro,romantic,attribute_value
0,androgynous,,,businesscasual,,,edgy,,,,,"androgynous, businesscasual, edgy"
1,androgynous,,boho,businesscasual,casual,,,,,,,"androgynous, boho, businesscasual, casual"
2,,athleisure,,,,,edgy,,,,,"athleisure, edgy"
3,,,boho,,casual,classic,,,,,,"boho, casual, classic"
4,,athleisure,,,,,edgy,,,,,"athleisure, edgy"


In [25]:
results_df['attribute_value'].value_counts()

androgynous, businesscasual, casual, edgy                                 2401
boho, casual, classic, glam, modern                                       1148
androgynous, businesscasual, edgy                                          616
casual, classic, edgy, glam                                                559
boho, casual, classic, glam                                                469
                                                                          ... 
businesscasual, classic, edgy                                                3
boho, businesscasual, glam                                                   3
athleisure, businesscasual, edgy                                             3
androgynous, boho, businesscasual, casual, classic, edgy, glam, modern       2
androgynous, casual, classic, edgy, modern                                   2
Name: attribute_value, Length: 92, dtype: int64